# Mutichannel MIP Preprocessing for Multiphases Data

A scipt that creates multichannel MIPs from multiphase data. Pulls in npz files, processes each npy (each phase) separately, then saves each phase into a directory. 

In [13]:
import numpy as np
import os
import sys
sys.path.append('../')
import etl.lib.transforms as transforms

In [14]:
def unzip(path):
    """
    unzips one npz file given path of file
    :param path: string path of a singular npz file
    :return: the contents of the zip file
    """
    files = np.load(path)
    return files

In [15]:
def mip(arr):
    """
    applies multichannel mip preprocessing to one array
    :param arr: numpy array to be preprocessed
    :return: MIP-ed array
    """
    location = 'numpy/axial'
    cropped = transforms.crop_multichannel_axial_fa(arr, location)
    remove_extremes = transforms.remove_extremes(cropped)
    return transforms.mip_multichannel(remove_extremes)

In [16]:
def save(mipped_arr, path):
    """
    saves multichannel mip array to the corresponding directory, organized
    based on phase
    :param mipped_arr: numpy array, mipped
    :param path: string path, indicated dir where file should be saved
    :return: nothing
    """
    np.save(path, mipped_arr)

In [49]:
test_scan = '/research/rih-cs/datasets/elvo-multiphase/v0.1/positive/P108.npz'
npy_files = np.load(test_scan)
phase1 = npy_files['arr_0']
phase2 = npy_files['arr_1']
phase3 = npy_files['arr_2']

# phase1 = mip(npy_files['arr_0'])
# phase2 = mip(npy_files['arr_1'])
# phase3 = mip(npy_files['arr_2'])

In [50]:
print(phase1.shape)
print(phase2.shape)
print(phase3.shape)

(1416, 230, 230)
(864, 176, 174)
(864, 176, 162)


In [42]:
location = 'numpy/axial'

cropped1 = transforms.crop_multichannel_axial_fa(npy_files['arr_0'], location)
cropped2 = transforms.crop_multichannel_axial_fa(npy_files['arr_1'], location)
cropped3 = transforms.crop_multichannel_axial_fa(npy_files['arr_2'], location)

In [45]:
print(cropped1.shape)

(3, 25, 230, 230)


In [51]:
test_mip = transforms.mip_multichannel(cropped1)

In [40]:
# print(phase1)
# print(phase2)
# print(phase3)

# print(cropped1)
# print(cropped2)
# print(cropped3)

print(test_mip)

[[[-992. -988. -986. ... -991. -992. -992.]
  [-992. -982. -987. ... -990. -993. -990.]
  [-988. -988. -987. ... -994. -994. -990.]
  ...
  [-993. -993. -991. ... -986. -987. -989.]
  [-997. -994. -993. ... -985. -989. -991.]
  [-996. -994. -997. ... -990. -986. -987.]]

 [[-991. -988. -980. ... -991. -992. -992.]
  [-990. -989. -986. ... -991. -993. -990.]
  [-988. -988. -992. ... -994. -993. -990.]
  ...
  [-992. -993. -989. ... -986. -987. -989.]
  [-992. -991. -992. ... -985. -989. -991.]
  [-992. -991. -992. ... -990. -986. -987.]]

 [[-990. -987. -978. ... -995. -991. -992.]
  [-990. -989. -986. ... -994. -993. -991.]
  [-989. -988. -991. ... -994. -993. -990.]
  ...
  [-992. -991. -989. ... -986. -986. -989.]
  [-991. -991. -992. ... -985. -989. -991.]
  [-989. -991. -990. ... -990. -986. -987.]]]


In [17]:
def preprocess(dir_path, category):
    """
    preprocesses all data in a given directory (all pos or all neg) and saves
    them to the correct location
    :param dir_path: path to the directory
    :param category: string, pos or neg
    :return: nothing
    """
    for npz_file in os.listdir(dir_path):
        assert npz_file.endswith(".npz")  # expecting npz files
        if npz_file.endswith(".npz"):  # ignore gstmp files
            patient_id = os.path.splitext(npz_file)[0]    # get id w/o extension

            # npy_files = unzip(dir_path + '/' + npz_file)  # unzip file
            npy_files = np.load(dir_path + '/' + npz_file)
            # assert len(npy_files) == 3                    # expect 3 phases

            try:
                phase1 = mip(npy_files['arr_0'])
            except Exception:
                print('could not mip phase 1 of ' + patient_id + ' in ' + category)

            try:
                phase2 = mip(npy_files['arr_1'])
            except Exception:
                print('could not mip phase 2 of ' + patient_id + ' in ' + category)

            try:
                phase3 = mip(npy_files['arr_2'])
            except Exception:
                print('could not mip phase 3 of ' + patient_id + ' in ' + category)